In [228]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Visualization
from sklearn.preprocessing import MinMaxScaler
import torch # Library for implementing Deep Neural Network
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
from copy import deepcopy as dc
# import ray
# from ray import tune
# from ray.tune.schedulers import ASHAScheduler

random.seed(42)

# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [229]:
# Loading the Apple.Inc Stock Data
import yfinance as yf
from datetime import date, timedelta, datetime

end_date = date.today().strftime("%Y-%m-%d") #end date for our data retrieval will be current date
start_date = '1990-01-01' # Beginning date for our historical data retrieval

df = yf.download('AAPL', start=start_date, end=end_date)# Function used to fetch the data

data = df[["Close"]]
data



[*********************100%%**********************]  1 of 1 completed


,Close
Date,
1990-01-02,0.332589
1990-01-03,0.334821
1990-01-04,0.335938
1990-01-05,0.337054
1990-01-08,0.339286
...,...
2024-06-11,207.149994
2024-06-12,213.070007
2024-06-13,214.240005


In [230]:
def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)

    for i in range(1, n_steps+1):
        df[f'Close(t-{i})'] = df['Close'].shift(i)

    df.dropna(inplace=True)

    return df

lookback = 50
shifted_df = prepare_dataframe_for_lstm(data, lookback)

shifted_df["MA50"] = shifted_df.iloc[:, 1:].mean(axis=1)
shifted_df["MA20"] = shifted_df.iloc[:, 1:21].mean(axis=1)

shifted_df

,Close,Close(t-1),Close(t-2),Close(t-3),Close(t-4),Close(t-5),Close(t-6),Close(t-7),Close(t-8),Close(t-9),...,Close(t-43),Close(t-44),Close(t-45),Close(t-46),Close(t-47),Close(t-48),Close(t-49),Close(t-50),MA50,MA20
Date,,,,,,,,,,,,,,,,,,,,,
1990-03-14,0.330357,0.329241,0.327009,0.329241,0.328125,0.315848,0.314732,0.308036,0.301339,0.305804,...,0.308036,0.321429,0.335938,0.339286,0.337054,0.335938,0.334821,0.332589,0.309509,0.309040
1990-03-15,0.328125,0.330357,0.329241,0.327009,0.329241,0.328125,0.315848,0.314732,0.308036,0.301339,...,0.308036,0.308036,0.321429,0.335938,0.339286,0.337054,0.335938,0.334821,0.309464,0.310156
1990-03-16,0.359375,0.328125,0.330357,0.329241,0.327009,0.329241,0.328125,0.315848,0.314732,0.308036,...,0.305804,0.308036,0.308036,0.321429,0.335938,0.339286,0.337054,0.335938,0.309330,0.311272
1990-03-19,0.378348,0.359375,0.328125,0.330357,0.329241,0.327009,0.329241,0.328125,0.315848,0.314732,...,0.311384,0.305804,0.308036,0.308036,0.321429,0.335938,0.339286,0.337054,0.309799,0.313951
1990-03-20,0.369420,0.378348,0.359375,0.328125,0.330357,0.329241,0.327009,0.329241,0.328125,0.315848,...,0.296875,0.311384,0.305804,0.308036,0.308036,0.321429,0.335938,0.339286,0.310625,0.317801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-11,207.149994,193.119995,196.889999,194.479996,195.869995,194.350006,194.029999,192.250000,191.289993,190.289993,...,167.779999,169.669998,168.449997,169.580002,168.820007,169.649994,168.839996,170.029999,179.966599,191.342498
2024-06-12,213.070007,207.149994,193.119995,196.889999,194.479996,195.869995,194.350006,194.029999,192.250000,191.289993,...,175.039993,167.779999,169.669998,168.449997,169.580002,168.820007,169.649994,168.839996,180.708999,192.385998
2024-06-13,214.240005,213.070007,207.149994,193.119995,196.889999,194.479996,195.869995,194.350006,194.029999,192.250000,...,176.550003,175.039993,167.779999,169.669998,168.449997,169.580002,168.820007,169.649994,181.593600,193.667999


In [231]:
shifted_df_as_numpy = shifted_df.to_numpy()

scaler = MinMaxScaler(feature_range=(-1, 1))
shifted_df_as_np = scaler.fit_transform(shifted_df_as_numpy)

shifted_df_as_np

X = shifted_df_as_np[:, 1:]
y = shifted_df_as_np[:, 0]

train_split = int(len(X) * 0.8)

X_train = X[:train_split]
X_test = X[train_split:]

y_train = y[:train_split]
y_test = y[train_split:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6904, 52), (1727, 52), (6904,), (1727,))

In [232]:
X_train = X_train.reshape((-1, lookback+2, 1))
X_test = X_test.reshape((-1, lookback+2, 1))
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

X_train shape: (6904, 52, 1)
X_test shape: (1727, 52, 1)


((6904, 52, 1), (1727, 52, 1), (6904, 1), (1727, 1))

In [233]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([6904, 52, 1]),
 torch.Size([1727, 52, 1]),
 torch.Size([6904, 1]),
 torch.Size([1727, 1]))

In [234]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

train_dataset

In [235]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [236]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([16, 52, 1]) torch.Size([16, 1])


In [237]:
X_train.shape

torch.Size([6904, 52, 1])

In [238]:
class StackedLSTM_CNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, conv_out_channels, kernel_size=3, pool_size=2, dropout=0.3):
        super(StackedLSTM_CNN, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.relu_layers = nn.ModuleList()
        self.pool_layers = nn.ModuleList()
        self.lstm_layers = nn.ModuleList()
        self.dropout_layers = nn.ModuleList()

        for i, hidden_size in enumerate(hidden_sizes):
            # Convolutional layer
            self.conv_layers.append(nn.Conv1d(in_channels=1 if i == 0 else conv_out_channels[i],
                                              out_channels=conv_out_channels[i],
                                              kernel_size=kernel_size))
            # ReLU activation layer
            self.relu_layers.append(nn.ReLU())
            # Pooling layer
            self.pool_layers.append(nn.MaxPool1d(kernel_size=pool_size))
            # LSTM layer
            self.lstm_layers.append(nn.LSTM(conv_out_channels, hidden_size, batch_first=True))
            # Dropout layer
            self.dropout_layers.append(nn.Dropout(dropout))

        # Final linear layer
        self.linear = nn.Linear(hidden_sizes[-1], 1)

    def forward(self, x):
        batch_size = x.size(0)
        outputs = x

        for conv, relu, pool, lstm, dropout in zip(self.conv_layers, self.relu_layers, self.pool_layers, self.lstm_layers, self.dropout_layers):
            # Apply convolutional layer
            outputs = conv(outputs.transpose(1, 2))  # Change shape for Conv1d
            # Apply ReLU activation
            outputs = relu(outputs)
            # Apply pooling layer
            outputs = pool(outputs)
            # Change shape back for LSTM
            outputs = outputs.transpose(1, 2)
            # Apply LSTM layer
            outputs, _ = lstm(outputs)  # Discard cell states for simplicity
            # Apply dropout
            outputs = dropout(outputs)

        outputs = outputs[:, -1, :]
        return self.linear(outputs)

# Example Usage
model1 = StackedLSTM_CNN(X_train.shape[1], hidden_sizes=[40, 25, 12, 6], conv_out_channels=[40, 25, 12, 6])
model1.to(device)
model1

StackedLSTM_CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 25, kernel_size=(3,), stride=(1,))
    (1-3): 3 x Conv1d(25, 25, kernel_size=(3,), stride=(1,))
  )
  (relu_layers): ModuleList(
    (0-3): 4 x ReLU()
  )
  (pool_layers): ModuleList(
    (0-3): 4 x MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm_layers): ModuleList(
    (0): LSTM(25, 40, batch_first=True)
    (1): LSTM(25, 25, batch_first=True)
    (2): LSTM(25, 12, batch_first=True)
    (3): LSTM(25, 6, batch_first=True)
  )
  (dropout_layers): ModuleList(
    (0-3): 4 x Dropout(p=0.3, inplace=False)
  )
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [239]:
class StackedLSTM(nn.Module):
  def __init__(self, input_size, hidden_sizes, dropout=0.3):
    super(StackedLSTM, self).__init__()
    self.lstm_layers = nn.ModuleList()
    self.dropout_layers = nn.ModuleList()

    # Define LSTM layers with decreasing hidden size
    for i, hidden_size in enumerate(hidden_sizes):
      self.lstm_layers.append(nn.LSTM(input_size if i == 0 else hidden_sizes[i-1], hidden_size))
      self.dropout_layers.append(nn.Dropout(dropout))

    # Final linear layer
    self.linear = nn.Linear(hidden_sizes[-1], 1)

  def forward(self, x):
    batch_size = x.size(0)
    outputs = x.transpose(1,2)

    for lstm, dropout in zip(self.lstm_layers, self.dropout_layers):
      outputs, _ = lstm(outputs)  # Discard cell states for simplicity
      outputs = dropout(outputs)

    outputs = outputs[:, -1, :]
    return self.linear(outputs)

# Example Usage
model = StackedLSTM(X_train.shape[1], hidden_sizes=[26])
model.to(device)
model

StackedLSTM(
  (lstm_layers): ModuleList(
    (0): LSTM(52, 26)
  )
  (dropout_layers): ModuleList(
    (0): Dropout(p=0.3, inplace=False)
  )
  (linear): Linear(in_features=26, out_features=1, bias=True)
)

In [240]:
X_train.shape[1]

52

In [241]:
learning_rate = 0.001
num_epochs = 100
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [242]:
def train_one_epoch(model):
    model.train(True)
    print(f'Epoch: {epoch + 1}')

    running_loss = 0.0  # Initialize running_loss

    train_loader_tmp = dc(train_loader)

    for batch_index, batch in enumerate(train_loader_tmp):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Training Loss: {running_loss / len(train_loader)}')

def validate_one_epoch(model):
    model.eval()
    running_loss = 0.0

    test_loader_tmp = dc(test_loader)

    for batch_index, batch in enumerate(test_loader_tmp):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.inference_mode():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [ ]:
print("LSTM:")
for epoch in range(num_epochs):
    train_one_epoch(model=model)
    validate_one_epoch(model=model)

print("LSTM-CNN:")
for epoch in range(num_epochs):
    train_one_epoch(model=model1)
    validate_one_epoch(model=model1)

In [ ]:
def plot(model):
  test_predictions = model(X_test.to(device)).detach().cpu().numpy().flatten()

  dummies = np.zeros((X_test.shape[0], lookback+3))
  dummies[:, 0] = test_predictions
  dummies = scaler.inverse_transform(dummies)

  test_predictions = dc(dummies[:, 0])
  test_predictions
  dummies = np.zeros((X_test.shape[0], lookback+3))
  dummies[:, 0] = y_test.flatten()
  dummies = scaler.inverse_transform(dummies)

  new_y_test = dc(dummies[:, 0])
  new_y_test
  plt.plot(new_y_test, label='Actual Close')
  plt.plot(test_predictions, label='Predicted Close')
  plt.xlabel('Day')
  plt.ylabel('Close')
  plt.legend()
  plt.show()

In [ ]:
print("LSTM:")
plot(model)
print("LSTM-CNN:")
plot(model1)